# Grid search MDF dataset

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from keras.utils import plot_model
from keras.models import Sequential
from keras.layers import Dense, Input, Embedding, Flatten, Concatenate, Lambda
from keras.optimizers import Adam

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_validate, train_test_split
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import MinMaxScaler

import train_utils
from NeuMF import NeuMF
from ECAM_NeuMF import ECAM_NeuMF
from moveCARS import moveCARS
from kerashypetune import KerasGridSearch

from implicit.evaluation import train_test_split as implicit_train_test_split
from implicit.als import AlternatingLeastSquares
from scipy.sparse import coo_matrix, csr_matrix
from tqdm import tqdm

pd.options.display.max_columns = 1000
import warnings
warnings.filterwarnings("ignore")

## Open Dataset

In [ ]:
def load_dataset(name : str):
    if dataset == 'mdf':
        df = pd.read_csv('Datasets/MDF_social/MDF_with_social_features.csv')
        df = df.drop(columns='time')
        df = df.drop_duplicates()
        df = df.reset_index(drop=True)
        context_labels = list(df.columns[3:66])
        item_labels = list(df.columns[66:92])
        user_labels = list(df.columns[92:106])
        social_labels = list(df.columns[106:])

    elif dataset == 'frappe':
        df = pd.read_csv('Datasets/frappe dataset/frappe_final.csv')
        context_labels = list(df.columns[3:27])
        item_labels = list(df.columns[27:54])
        user_labels = list(df.columns[54:])
        social_labels = []
    return df, user_labels, item_labels, context_labels, social_labels

In [ ]:
dataset = 'mdf'
df, user_labels, item_labels, context_labels, social_labels = load_dataset(dataset)

n_users = df.user.nunique()
n_items = df.item.nunique()
n_contexts = len(context_labels)

print(f"rating with value 1: {df[df.rating == 1]['rating'].count() * 100 / len(df)} %")
print(f"users: {n_users} \t items: {n_items} \t rating: {len(df)}")
print(f"user_features: {len(user_labels)} \t items_features: {len(item_labels)} \t social_features: {len(social_labels)} \t contexts_features: {n_contexts} \t ")

## Grid search

In [ ]:
param_grid = {
    'n_users': n_users,
    'n_items': n_items,
    'n_contexts': n_contexts,
    'epochs': [5, 10, 15, 20], 
    'batch_size': [64, 128, 256],
    'learn_rate': [0.0001, 0.001, 0.01]
}
open('grid_search_result.txt', 'w').close()

### ALS matrix factorization

In [ ]:
grid = ParameterGrid({'factors': [64, 128, 256], 'regularization': [0.01, 0.1, 1, 5, 7, 10], 'iterations':[1, 10, 50]})

ratings = coo_matrix((df['rating'].astype(np.float32),
                     (df['item'],
                      df['user']))).tocsr()
train, test = implicit_train_test_split(ratings, train_percentage=0.80)

best_params = {}
best_AUC = 0
for params in tqdm(grid):
    auc = 0
    for split in range(5):
        model = AlternatingLeastSquares(**params)
        model.fit(train, show_progress=False)
        auc = auc + train_utils.mf_AUC(model, train, test)
    auc = auc / 5
    if auc > best_AUC:
        best_params = params
        best_AUC = auc
print(best_AUC, best_params)

### NeuMF

In [ ]:
print("grid search on NeuMF...")
x_train, x_test, y_train, y_test = train_test_split(df[['user', 'item']], df['rating'], test_size=0.20, random_state=42)
kgs = KerasGridSearch(NeuMF, param_grid, monitor='val_auc', greater_is_better=True, tuner_verbose=0)
kgs.search([x_train.user, x_train.item], y_train, validation_data=([x_test.user, x_test.item], y_test))
print(f'NeuMF best AUC: {kgs.best_score} using {kgs.best_params}', file=open("grid_search_result.txt", "a"))
print("Done!")

### ECAM NeuMF

In [ ]:
print("grid search on ECAM NeuMF...")
x_train, x_test, y_train, y_test = train_test_split(df[['user', 'item'] + context_labels], df['rating'], test_size=0.20, random_state=42)
kgs = KerasGridSearch(ECAM_NeuMF, param_grid, monitor='val_auc', greater_is_better=True, tuner_verbose=0)
kgs.search([x_train.user, x_train.item, x_train[context_labels]], y_train, validation_data=([x_test.user, x_test.item, x_test[context_labels]], y_test))
print(f'ECAM NeuMF best AUC: {kgs.best_score} using {kgs.best_params}', file=open("grid_search_result.txt", "a"))
print("Done!")

### moveCARS

In [ ]:
print("grid search on moveCARS...")

x = df[item_labels + context_labels + user_labels]
y = df['rating']

model = KerasClassifier(build_fn=moveCARS, verbose=False)

param_grid = {
    'learn_rate': [0.0001, 0.001, 0.005, 0.01],
    'batch_size': [64, 128, 256],
    'epochs': [5, 10, 15, 20, 30], 
    'layers': [3],
    'neurons': [100, 200]
}

grid = GridSearchCV(estimator=model, scoring=['accuracy', 'roc_auc'], refit='roc_auc', param_grid=param_grid, 
                    cv=KFold(shuffle=True, n_splits=5, random_state=42), verbose=True)
grid_results = grid.fit(x, y)

mean_accuracy = grid_results.cv_results_['mean_test_accuracy']
mean_auc = grid_results.cv_results_['mean_test_roc_auc']
params = grid_results.cv_results_['params']

print(f'FFnet best AUC: {grid_results.best_score_} using {grid_results.best_params_}', file=open("grid_search_result.txt", "a"))
print("Done!")

## moveCARS on different features

In [ ]:
param_grid = {
    'learn_rate': [0.0001, 0.001, 0.005, 0.01],
    'batch_size': [64, 128, 256],
    'epochs': [5, 10, 15, 20, 30], 
    'layers': [3],
    'neurons': [100]
}

open('grid_search_result_features.txt', 'w').close()

train_labels = [item_labels+user_labels, 
                item_labels+user_labels+social_labels, 
                item_labels+user_labels+context_labels,
                item_labels+user_labels+context_labels+social_labels]

for idx, x_labels in enumerate(train_labels):
    print('-'*10 + f" Starting grid search {idx} " + '-'*10)
    x = df[x_labels]     # get only some features
    y = df['rating']     # the value to predict is always the same
    model = KerasClassifier(build_fn=moveCARS, verbose=False)
    # create and fit gridsearch
    grid = GridSearchCV(estimator=model, scoring=['accuracy', 'roc_auc'], refit='roc_auc', param_grid=param_grid, 
                        cv=KFold(shuffle=True, n_splits=2, random_state=42), verbose=True)
    grid_results = grid.fit(x, y)

    mean_accuracy = grid_results.cv_results_['mean_test_accuracy']
    mean_auc = grid_results.cv_results_['mean_test_roc_auc']
    params = grid_results.cv_results_['params']

    print(f'best AUC: {grid_results.best_score_} using {grid_results.best_params_}', file=open("grid_search_result_features.txt", "a"))

print("Done!")